In [6]:
pip install --user -U nltk

distutils: /Users/juliette/Library/Python/3.9/lib/python/site-packages
sysconfig: /Users/juliette/Library/Python/3.9/lib/python3.9/site-packages
distutils: /Users/juliette/Library/Python/3.9/lib/python/site-packages
sysconfig: /Users/juliette/Library/Python/3.9/lib/python3.9/site-packages
user = True
home = None
root = None
prefix = None
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.3.2/libexec/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install gensim

     |████████████████████████████████| 24.0 MB 3.7 MB/s            
     |████████████████████████████████| 56 kB 3.1 MB/s            
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.3.2/libexec/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [43]:
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /Users/juliette/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/juliette/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/juliette/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
df_train = pd.read_json("train.jsonl",lines=True)
df_test = pd.read_json("test.jsonl",lines=True)
df_dev = pd.read_json("dev.jsonl",lines=True)

In [9]:
df_train.head()

,id,img,label,text
0,42953,img/42953.png,0,its their character not their color that matters
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...
2,13894,img/13894.png,0,putting bows on your pet
3,37408,img/37408.png,0,i love everything and everybody! except for sq...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h..."


# EDA

In [17]:
#Distribution : 
x=df_train['label'].value_counts()
print(x)

0    5450
1    3050
Name: label, dtype: int64


Il y a 5450 memes non haineux et 3050 memes haineux

In [18]:
# Vérification des NA
df_train.isna().sum()

id       0
img      0
label    0
text     0
dtype: int64

Il n'y a aucune valeur manquante

In [19]:
# Nombre de mots 

df_train['word_count'] = df_train['text'].apply(lambda x: len(str(x).split()))
print(df_train[df_train['label']==1]['word_count'].mean()) # memes haineux
print(df_train[df_train['label']==0]['word_count'].mean()) # memes non haineux

12.79344262295082
11.154495412844037


Il y a environ le même nombre de mots entre les deux types de même (en moyenne un de plus pour les mêmes haineux)

In [20]:
# Nombre de lettres
df_train['char_count'] = df_train['text'].apply(lambda x: len(str(x)))
print(df_train[df_train['label']==1]['char_count'].mean()) #memes haineux
print(df_train[df_train['label']==0]['char_count'].mean()) #memes non haineux

68.48360655737704
58.489724770642205


Il y a environ une dizaine de caractères en plus dans les mêmes haineux

# Preparation de la base avant la classification

In [30]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/juliette/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [31]:
#On enlève la ponctuatoin et les strips
def preprocess(text):
    text = text.lower() 
    text=text.strip()  #suppression des caractères de tête et de queue
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

 
# STOPWORD REMOVAL : on enlève les mots inutiles "the" etc.
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

#LEMMATIZATION -> pour normaliser le texte
# On initialise le lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Tokenize 
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)



In [32]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))
df_train['clean_text'] = df_train['text'].apply(lambda x: finalpreprocess(x))
df_train.head()

,id,img,label,text,word_count,char_count,clean_text
0,42953,img/42953.png,0,its their character not their color that matters,8,48,character color matter
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...,12,58,afraid love everyone like ex
2,13894,img/13894.png,0,putting bows on your pet,5,24,put bow pet
3,37408,img/37408.png,0,i love everything and everybody! except for sq...,11,70,love everything everybody except squirrel hate...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h...",7,51,everybody love chocolate chip cooky even hitler


In [34]:
# Mise en vecteur (word embedding)

#SPLITTING le TRAINING DATASET EN UN TRAIN ET UN TEST
X_train, X_test, y_train, y_test = train_test_split(df_train["clean_text"],df_train["label"],test_size=0.2,shuffle=True)

#Word2Vec (on passe en vecteur pour obtenir des listes)
X_train_tok= [nltk.word_tokenize(i) for i in X_train]  
X_test_tok= [nltk.word_tokenize(i) for i in X_test]

In [38]:
#Tf-Idf
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) # on obtient des matrices
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

In [60]:
#building Word2Vec model
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
def fit(self, X, y):
        return self
def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

        df_train['clean_text_tok']=[nltk.word_tokenize(i) for i in df_train['clean_text']]   
        model = Word2Vec(df_train['clean_text_tok'],min_count=1) 
        w2v = dict(zip(model.wv.index_to_key, model.wv.vectors)) 

        modelw = MeanEmbeddingVectorizer(w2v)

# converting text to numerical data using Word2Vec
        X_train_vectors_w2v = modelw.transform(X_train_tok)
        X_test_vectors_w2v = modelw.transform(X_test_tok)

# Classification

In [61]:
#FITTING THE CLASSIFICATION MODEL using Logistic Regression(tf-idf)
lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_tfidf.fit(X_train_vectors_tfidf, y_train)  
#Predict y value for test dataset
y_predict = lr_tfidf.predict(X_test_vectors_tfidf)
y_prob = lr_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))
 
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

              precision    recall  f1-score   support

           0       0.71      0.77      0.74      1107
           1       0.49      0.42      0.46       593

    accuracy                           0.65      1700
   macro avg       0.60      0.59      0.60      1700
weighted avg       0.64      0.65      0.64      1700

Confusion Matrix: [[848 259]
 [342 251]]
AUC: 0.6499769213543737


In [62]:
#FITTING THE CLASSIFICATION MODEL using Logistic Regression (W2v)
lr_w2v=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_w2v.fit(X_train_vectors_w2v, y_train)  #model

#Predict y value for test dataset
y_predict = lr_w2v.predict(X_test_vectors_w2v)
y_prob = lr_w2v.predict_proba(X_test_vectors_w2v)[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))
 
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

NameError: name 'X_train_vectors_w2v' is not defined